<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Reduce-Memory-size" data-toc-modified-id="Reduce-Memory-size-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Reduce Memory size</a></span></li><li><span><a href="#Missing-values-and-Duplicated-ID" data-toc-modified-id="Missing-values-and-Duplicated-ID-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Missing values and Duplicated ID</a></span></li><li><span><a href="#Features" data-toc-modified-id="Features-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Features</a></span><ul class="toc-item"><li><span><a href="#Vendor-id" data-toc-modified-id="Vendor-id-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Vendor id</a></span></li></ul></li><li><span><a href="#Latutudes-and-Longitudes" data-toc-modified-id="Latutudes-and-Longitudes-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Latutudes and Longitudes</a></span></li></ul></div>

# Description

The dataset is based on the [2016 NYC Yellow Cab trip record](https://cloud.google.com/bigquery/public-data/nyc-tlc-trips) and data was originally published by the [NYC Taxi and Limousine Commission (TLC)](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml). 

**Features**

- `id` - a unique identifier for each trip
- `vendor_id` - a code indicating the provider associated with the trip record
- `pickup_datetime` - date and time when the meter was engaged
- `dropoff_datetime` - date and time when the meter was disengaged
- `passenger_count` - the number of passengers in the vehicle (driver entered value)
- `pickup_longitude` - the longitude where the meter was engaged
- `pickup_latitude` - the latitude where the meter was engaged
- `dropoff_longitude` - the longitude where the meter was disengaged
- `dropoff_latitude` - the latitude where the meter was disengaged
- `store_and_fwd_flag` - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip
- `trip_duration` - duration of the trip in seconds





# Imports

In [2]:
import numpy as np
import pandas as pd
import os,sys,time

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# settings
SEED = 100
pd.set_option('max_columns',100)
pd.set_option('plotting.backend','matplotlib') # matplotlib, bokeh, altair, plotly

%matplotlib inline
%load_ext watermark
%watermark -iv

numpy   1.19.5
seaborn 0.11.0
pandas  1.1.4



In [3]:
# my local library
import sys
sys.path.append("/Users/poudel/Dropbox/a00_Bhishan_Modules/")
sys.path.append("/Users/poudel/Dropbox/a00_Bhishan_Modules/bhishan")
from bhishan import bp

# Load the data

In [4]:
!ls ../data
!du -sh ../data/train.zip

train.zip
 63M	../data/train.zip


In [5]:
df = pd.read_csv('../data/train.zip',compression='zip',
                 infer_datetime_format=True,parse_dates=[2,3],
                engine='c')
print(f"rows = {df.shape[0]:,d} columns = {df.shape[1]:,d}")
df.head(2).append(df.tail(2))

rows = 1,458,644 columns = 11


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373
1458643,id1209952,1,2016-04-05 14:44:25,2016-04-05 14:47:43,1,-73.979538,40.781750,-73.972809,40.790585,N,198


In [6]:
df.bp.describe()

,Feature,Type,N,Count,Unique,Missing,MissingPct,Zeros,ZerosPct,Ones,OnesPct,mean,std,min,max,25%,50%,75%,Feature2,smallest5,largest5,first5,last5
4,passenger_count,int64,1458644,1458644,10,0,0.00,60,0.00,1033540,70.86,1.66,1.31,0.00,9.00,1.00,1.00,2.00,passenger_count,"[0, 0, 0, 0, 0]","[9, 8, 7, 7, 7]","[1, 1, 1, 1, 1]","[4, 1, 1, 1, 1]"
0,id,object,1458644,1458644,1458644,0,0.00,0,0.00,0,0.00,,,,,,,,id,"['id0000001', 'id0000003', 'id0000005', 'id0000008', 'id0000009']","['id4000000', 'id3999999', 'id3999998', 'id3999992', 'id3999988']","['id2875421', 'id2377394', 'id3858529', 'id3504673', 'id2181028']","['id2376096', 'id1049543', 'id2304944', 'id2714485', 'id1209952']"
1,vendor_id,int64,1458644,1458644,2,0,0.00,0,0.00,678342,46.50,1.53,0.50,1.00,2.00,1.00,2.00,2.00,vendor_id,"[1, 1, 1, 1, 1]","[2, 2, 2, 2, 2]","[2, 1, 2, 2, 2]","[2, 1, 2, 1, 1]"
2,pickup_datetime,datetime64[ns],1458644,1458644,1380222,0,0.00,0,0.00,0,0.00,,,,,,,,pickup_datetime,"[Timestamp('2016-01-01 00:00:17'), Timestamp('2016-01-01 00:00:53'), Timestamp('2016-01-01 00:01:01'), Timestamp('2016-01-01 00:01:14'), Timestamp('2016-01-01 00:01:20')]","[Timestamp('2016-06-30 23:59:39'), Timestamp('2016-06-30 23:59:37'), Timestamp('2016-06-30 23:59:10'), Timestamp('2016-06-30 23:59:09'), Timestamp('2016-06-30 23:58:52')]","[Timestamp('2016-03-14 17:24:55'), Timestamp('2016-06-12 00:43:35'), Timestamp('2016-01-19 11:35:24'), Timestamp('2016-04-06 19:32:31'), Timestamp('2016-03-26 13:30:55')]","[Timestamp('2016-04-08 13:31:04'), Timestamp('2016-01-10 07:35:15'), Timestamp('2016-04-22 06:57:41'), Timestamp('2016-01-05 15:56:26'), Timestamp('2016-04-05 14:44:25')]"
3,dropoff_datetime,datetime64[ns],1458644,1458644,1380377,0,0.00,0,0.00,0,0.00,,,,,,,,dropoff_datetime,"[Timestamp('2016-01-01 00:03:31'), Timestamp('2016-01-01 00:05:54'), Timestamp('2016-01-01 00:07:13'), Timestamp('2016-01-01 00:07:25'), Timestamp('2016-01-01 00:07:49')]","[Timestamp('2016-07-01 23:02:03'), Timestamp('2016-07-01 22:10:26'), Timestamp('2016-07-01 20:17:00'), Timestamp('2016-07-01 19:54:54'), Timestamp('2016-07-01 16:37:39')]","[Timestamp('2016-03-14 17:32:30'), Timestamp('2016-06-12 00:54:38'), Timestamp('2016-01-19 12:10:48'), Timestamp('2016-04-06 19:39:40'), Timestamp('2016-03-26 13:38:10')]","[Timestamp('2016-04-08 13:44:02'), Timestamp('2016-01-10 07:46:10'), Timestamp('2016-04-22 07:10:25'), Timestamp('2016-01-05 16:02:39'), Timestamp('2016-04-05 14:47:43')]"
5,pickup_longitude,float64,1458644,1458644,23047,0,0.00,0,0.00,0,0.00,-73.97,0.07,-121.93,-61.34,-73.99,-73.98,-73.97,pickup_longitude,"[-121.93334197998048, -121.93323516845705, -79.56973266601562, -79.48789978027342, -78.54740142822266]","[-61.33552932739258, -65.848388671875, -65.89738464355469, -66.97216033935545, -68.77843475341797]","[-73.98215484619139, -73.98041534423827, -73.97902679443358, -74.01004028320312, -73.97305297851562]","[-73.9822006225586, -74.00094604492188, -73.9591293334961, -73.9820785522461, -73.9795379638672]"
6,pickup_latitude,float64,1458644,1458644,45245,0,0.00,0,0.00,0,0.00,40.75,0.03,34.36,51.88,40.74,40.75,40.77,pickup_latitude,"[34.359695434570305, 34.71223449707032, 35.0815315246582, 35.310306549072266, 36.02930068969727]","[51.88108444213867, 44.37194442749024, 43.91176223754883, 43.48688507080078, 43.13965225219727]","[40.76793670654297, 40.738563537597656, 40.763938903808594, 40.719970703125, 40.79320907592773]","[40.745521545410156, 40.74737930297852, 40.768798828125, 40.749061584472656, 40.7817497253418]"
7,dropoff_longitude,float64,1458644,1458644,33821,0,0.00,0,0.00,0,0.00,-73.97,0.07,-121.93,-61.34,-73.99,-73.98,-73.96,dropoff_longitude,"[-121.9333038330078, -121.93320465087892, -80.35543060302734, -79.81797790527342, -79.7861328125]","[-61.33552932739258, -65.848388671875, -65.89738464355469, -68.77843475341797, -69.04801940917969]","[-73.96463012695312, -73.99948120117188, -74.00533294677734, -74.01226806640625, -73.9729232788086]","[-73.99491119384766, -73.97018432617188, -74.00443

# Reduce Memory size

In [7]:
df.memory_usage().sum() / 2**20 # MB

122.41439819335938

In [8]:
df.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')

In [9]:
df.apply(pd.Series.nunique).loc[lambda x: x<20]

vendor_id              2
passenger_count       10
store_and_fwd_flag     2
dtype: int64

In [10]:
cols_cat = df.apply(pd.Series.nunique).loc[lambda x: x<20].index.values
cols_cat

array(['vendor_id', 'passenger_count', 'store_and_fwd_flag'], dtype=object)

In [11]:
for c in cols_cat:
    df[c] = df[c].astype('category')

In [12]:
# integers
cols_int = df.select_dtypes('integer').columns.tolist()
cols_int

['trip_duration']

In [13]:
for c in cols_int:
    df[c] = df[c].astype(np.int32)

In [14]:
# floats
cols_float = df.select_dtypes('float').columns.tolist()
cols_float

['pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude']

In [15]:
for c in cols_float:
    df[c] = df[c].astype('float32')

In [16]:
df.memory_usage().sum() / 2**30 # MB

0.06384867057204247

# Missing values and Duplicated ID

In [17]:
df.isna().sum().sum()

0

In [18]:
df.head(2)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663


In [19]:
# check for duplicates
df[df.duplicated('id')]

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration


# Features

## Vendor id

In [20]:
df['vendor_id'].unique()

[2, 1]
Categories (2, int64): [2, 1]

# Latutudes and Longitudes

In [21]:
df[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].agg(['min','max'])

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
min,-121.933342,34.359695,-121.933304,32.181141
max,-61.335529,51.881084,-61.335529,43.921028
